In [11]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

# load general packages
import numpy as np
import matplotlib.pyplot as plt
from consav.linear_interp import interp_4d

# load modules related to this project
from educationmodel import EducationModel
import estimate as est 

colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.alpha":"0.25", "grid.linestyle": "--"})
plt.rcParams.update({'font.size': 14})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [12]:
model = EducationModel() # Define the model

In [ ]:
model.solve() # Solve the model

# Redefine
sol = model.sol 
par = model.par 


## Simulate Model

In [ ]:
# Simulate model
model.simulate()
sim=model.sim

In [ ]:
# Determining the different types
Type1 = np.where((sim.util_school_fix == 50) & (sim.abil_work_fix == 30),1,0) # Low utility school, low ability work
Type2 = np.where((sim.util_school_fix == 200) & (sim.abil_work_fix == 30),1,0) # High utility school, low ability work
Type3 = np.where((sim.util_school_fix == 50) & (sim.abil_work_fix == 120),1,0) # Low utility school, high ability work
Type4 = np.where((sim.util_school_fix == 200) & (sim.abil_work_fix == 120),1,0) # High utility school, high ability work


In [ ]:
mean_school_vectors =[]
mean_interrupt_vectors =[]
mean_school_time_vectors = []
mean_wage_vectors = []
mean_experience_vectors = []

In [ ]:
types = [Type1,Type2,Type3,Type4]
for s in types:
    i = np.where(s == 1)[0]

    mean_school = sim.school[i].mean(axis=0)
    mean_interrupt = sim.interrupt[i].mean(axis=0)
    mean_school_time = sim.school_time[i].mean(axis=0)
    mean_wage = np.nanmean(sim.wage[i],axis=0)
    mean_experience = np.nanmean(sim.experience[i],axis=0)
    mean_school_vectors.append(mean_school)
    mean_interrupt_vectors.append(mean_interrupt)
    mean_school_time_vectors.append(mean_school_time)
    mean_wage_vectors.append(mean_wage)
    mean_experience_vectors.append(mean_experience)


In [ ]:
plot_vectors = [mean_school_vectors, mean_interrupt_vectors, mean_school_time_vectors, mean_wage_vectors, mean_experience_vectors]
vector_names = ["Mean schooling choice","Mean interruption choice", "Mean school time", "Mean wage", "Mean experience"]

In [ ]:
for vector, name in zip(plot_vectors, vector_names):
    for i in range(4):
        plt.plot(vector[i], label=f"{name} for type {i+1}")
    plt.legend()
    plt.xlabel("Time period")
    plt.ylabel(name)
    plt.savefig(f"{name}_for_type_{i+1}_over_time.png") # Save the figure
    plt.show()


## Estimate the parameters

In [13]:
estimate_ = est.estimate_class() # define the estimate function
family_data, decision_data = estimate_.read_data() # Read data

In [ ]:
print(f'Max util_school: {util_school.max()}, Min util_school: {util_school.min()}')
print(f'Max ability_wage: {ability_wage.max()}, Min ability_wage: {ability_wage.min()}')

In [ ]:
delta7= np.linspace(-5,5,5)
log_lik = np.nan + np.zeros(5)

In [ ]:
for i in range(5):
    pnames = ['delta7']
    theta = np.array([delta7[i]])
    log_lik[i]=estimate_.ll(theta,model, family_data, decision_data, pnames)

In [ ]:
fig = plt.figure(figsize=(6,4),dpi=100)

ax = fig.add_subplot(1,1,1)
ax.set_title('Log likelihood')

ax.plot(delta7,log_lik)

ax.set_xlabel('delta7')
ax.set_ylabel('log likelihood')


fig.tight_layout()

In [14]:
pnames = ['delta7','delta8']
theta0 = [-1.09911098,-0.02504128]


In [15]:
res = estimate_.estimate(model, family_data, decision_data, pnames, theta0)

delta7 -1.0979559481181265
delta8 -0.0001
delta7 -1.0979559381181265
delta8 -0.0001
delta7 -1.0979559481181265
delta8 -9.999000000000001e-05
delta7RUNNING THE L-BFGS-B CODE
 -1.1612748790954819
delta8 -0.9980933431540938

           * * *

Machine precision = 2.220D-16
 N =            2     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  1.68867D+04    |proj g|=  6.30094D+04


 This problem is unconstrained.


delta7 -1.161274869095482
delta8 -0.9980933431540938
delta7 -1.1612748790954819
delta8 -0.9980933331540938
delta7 -1.1079860984078287
delta8 -0.15818894852532903
delta7 -1.1079860884078288
delta8 -0.15818894852532903
delta7 -1.1079860984078287
delta8 -0.15818893852532903
delta7 -1.0998750715409742
delta8 -0.030348021738998587
delta7 -1.0998750615409743
delta8 -0.030348021738998587
delta7 -1.0998750715409742
delta8 -0.030348011738998586
delta7
 -1.0995818440378389
delta8 -0.027492334451587108
At iterate    1    f=  1.61381D+04    |proj g|=  6.56996D+03
delta7 -1.099581834037839
delta8 -0.027492334451587108
delta7 -1.0995818440378389
delta8 -0.027492324451587107
delta7
 -1.09921644169869
delta8 -0.02489288506457923
At iterate    2    f=  1.61254D+04    |proj g|=  3.12984D+03
delta7 -1.09921643169869
delta8 -0.02489288506457923
delta7 -1.09921644169869
delta8 -0.02489287506457923
delta7
 -1.0991109776100523
delta8 -0.025041280128792913
At iterate    3    f=  1.61199D+04    |proj g|=  2.00

/Users/HP/Documents/Skole/Uni/Dynamic Programming/Term paper/Dynamic-Programming/estimate.py:189: RuntimeWarning: invalid value encountered in sqrt
  se = np.sqrt(np.diag(cov))


In [16]:
res

{'theta': array([-1.09911098, -0.02504128]),
 'se': array([0.01071953,        nan]),
 't': array([-102.53353081,           nan]),
 'cov': array([[ 1.14908264e-04,  2.15191696e-07],
        [ 2.15191696e-07, -7.82279539e-08]]),
 'success': True,
 'nit': 4,
 'nfev': 21,
 'fun': 16119.838749521497}

In [38]:
cov, se = estimate_.variance(model,['delta7','delta8'],family_data,decision_data,np.array([-1.09911098,-0.02504128]))

basis [-1.09911098 -0.02504128]
fhandle <function estimate_class.variance.<locals>.<lambda> at 0x7fae30999af0>
delta7 -1.09911098
delta8 -0.02504128
delta7 -1.0992208910980001
delta8 -0.02504128
delta7 -1.09911098
delta8 -0.025043784128
delta7 -1.0993308021960002
delta8 -0.02504128
delta7 -1.0992208910980001
delta8 -0.025043784128
delta7 -1.09911098
delta8 -0.025046288256


In [39]:
se

array([0.05258693, 0.00361157])

### Illustrate results

In [49]:
# Find the likelihood value for different combinations 
model_ = model.copy()

# Prøver for forskellige værdier af RC og c. 
Ndelta7 = 10
Ndelta8 = 10

log_lik = np.nan + np.zeros((Ndelta7,Ndelta8))
delta7= np.linspace(-1.3,0,Ndelta7)
delta8 = np.linspace(-1.3,1,Ndelta8)

for i in range(Ndelta7):
    for j in range(Ndelta8):
        pnames = ['delta7','delta8']
        theta = np.array([delta7[i], delta8[j]])
        log_lik[i,j]= estimate_.ll(theta, model_, family_data,decision_data,pnames)


delta7 -1.3
delta8 -1.3
delta7 -1.3
delta8 -1.0444444444444445
delta7 -1.3
delta8 -0.788888888888889
delta7 -1.3
delta8 -0.5333333333333334
delta7 -1.3
delta8 -0.2777777777777779
delta7 -1.3
delta8 -0.022222222222222365
delta7 -1.3
delta8 0.23333333333333317
delta7 -1.3
delta8 0.4888888888888887
delta7 -1.3
delta8 0.7444444444444442
delta7 -1.3
delta8 1.0
delta7 -1.1555555555555557
delta8 -1.3
delta7 -1.1555555555555557
delta8 -1.0444444444444445
delta7 -1.1555555555555557
delta8 -0.788888888888889
delta7 -1.1555555555555557
delta8 -0.5333333333333334
delta7 -1.1555555555555557
delta8 -0.2777777777777779
delta7 -1.1555555555555557
delta8 -0.022222222222222365
delta7 -1.1555555555555557
delta8 0.23333333333333317
delta7 -1.1555555555555557
delta8 0.4888888888888887
delta7 -1.1555555555555557
delta8 0.7444444444444442
delta7 -1.1555555555555557
delta8 1.0
delta7 -1.011111111111111
delta8 -1.3
delta7 -1.011111111111111
delta8 -1.0444444444444445
delta7 -1.011111111111111
delta8 -0.7888888

In [ ]:
# plot figure in three dimensions
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.style.use('seaborn-whitegrid')


fig = plt.figure(figsize=(10,5))
ax = fig.add_subplot(1,1,1,projection='3d', computed_zorder=False)

# Make data.
X, Y = np.meshgrid(RC, c,indexing='ij')
x, y = np.unravel_index(np.argmax(log_lik), log_lik.shape)

# Plot the surface.
surf = ax.plot_surface(X, Y,  , cmap=cm.jet)

#Plot max value
max = ax.scatter(RC[x], c[y], log_lik[x,y], color=['black'], marker='o', s=10)

# Customize the axis.
ax.set_xlabel(f'RC')
ax.set_ylabel(f'c')
ax.set_title(f'Log-likelihood (RC,c)')
ax.invert_xaxis()

# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()